In [52]:
%pylab inline
from tqdm import tqdm
import time,wave
import alsaaudio, time, audioop, sys, librosa,json
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pypot.dynamixel
import pypot.robot
from explauto.environment import environments


my_config ={}
my_config['controllers'] = {}
my_config['controllers']['upper_body_controler'] = {
    'port': '/dev/ttyUSB0',     # For Linux
    #'port': '/dev/tty.usbserial-AI03QEN0', # For OSX
    'sync_read': False,
    'attached_motors': ['arms'],
    'protocol': 1,
}
my_config['motorgroups']={}
my_config['motorgroups'] = {
    'arms': ['right_arm'],
    'right_arm': ['r_shoulder_x', 'r_shoulder_y', 'r_arm_z','r_elbow_y']
}
my_config['motors'] = {}
my_config['motors']['r_shoulder_y'] = {
    'id': 51,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (-50, 170),
}
my_config['motors']['r_shoulder_x'] = {
    'id': 52,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (-90, 0),
}
my_config['motors']['r_arm_z'] = {
    'id': 53,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (-20, 95),
}
my_config['motors']['r_elbow_y'] = {
    'id': 54,
    'type': 'MX-28',
    'orientation': 'indirect',
    'offset': 0.0,
    'angle_limit': (0, 130),
}

poppy = pypot.robot.from_config(my_config)

poppy.start_sync()
time.sleep(2)

for m in poppy.motors:
    m.compliant = False
    m.goto_behavior = 'minjerk'
pos = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []


# audio recording setup

w = wave.open("../in_Data/rattle.wav",'w')
# Open the device in nonblocking capture mode. The last argument could
# just as well have been zero for blocking mode. Then we could have
# left out the sleep call in the bottomvim of the loop
inp = alsaaudio.PCM(alsaaudio.PCM_CAPTURE,alsaaudio.PCM_NONBLOCK,device='sysdefault:CARD=C920')

inp.setchannels(2)
inp.setrate(88200)
inp.setformat(alsaaudio.PCM_FORMAT_GSM)

# The period size controls the internal number of frames per period.
# The significance of this parameter is documented in the ALSA api.
# For our purposes, it is suficcient to know that reads from the device
# will return this many frames. Each frame being 2 bytes long.
# This means that the reads below will return either 320 bytes of data
# or 0 bytes of data. The latter is possible because we are in nonblocking
# mode.
inp.setperiodsize(160)
w.setnchannels(2)
w.setsampwidth(2)
w.setframerate(44100)

data_table = []


Populating the interactive namespace from numpy and matplotlib


In [60]:
# Primitive actions

def rest_position():
    poppy.r_shoulder_x.goal_position = 0
    poppy.r_shoulder_y.goal_position = 0
    poppy.r_arm_z.goal_position = 0
    poppy.r_elbow_y.goal_position = 0

def app():
    pos.append(poppy.r_shoulder_x.present_position)
    pos.append(poppy.r_shoulder_y.present_position)
    pos.append(poppy.r_arm_z.present_position)
    pos.append(poppy.r_elbow_y.present_position)

    sys_load.append(poppy.r_shoulder_x.present_load)
    sys_load.append(poppy.r_shoulder_y.present_load)
    sys_load.append(poppy.r_arm_z.present_load)
    sys_load.append(poppy.r_elbow_y.present_load)
    
    speed.append(poppy.r_shoulder_x.present_speed)
    speed.append(poppy.r_shoulder_y.present_speed)
    speed.append(poppy.r_arm_z.present_speed)
    speed.append(poppy.r_elbow_y.present_speed)
    
    temp.append(poppy.r_shoulder_x.present_temperature)
    temp.append(poppy.r_shoulder_y.present_temperature)
    temp.append(poppy.r_arm_z.present_temperature)
    temp.append(poppy.r_elbow_y.present_temperature)
     
    volt.append(poppy.r_shoulder_x.present_voltage)
    volt.append(poppy.r_shoulder_y.present_voltage)
    volt.append(poppy.r_arm_z.present_voltage)
    volt.append(poppy.r_elbow_y.present_voltage)
    
def hand_wave(sec=10, rest=0.5):
    t0 = time.time()
    while True:
        
        t1 = time.time()
        if t1-t0 >= sec:
            break

        # run for sec
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = -70
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -170  
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 60;
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 125
        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
    
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = -50
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = -135
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 75
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = 90
        
        app()
        TIME.append(t1-t0)

        time.sleep(rest)
        app()

def rattle_shake(sec=10, rest=0.42):
    t0 = time.time()
    while True:
        
        t1 = time.time()
        if t1-t0 >= sec:
            break

        # run for sec
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = -25 #-25
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = 0  #0
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 0; #0
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = -10 #-10
        
        app()
        TIME.append(t1-t0)
        time.sleep(rest)
        
    
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_y.goal_position = -45 #-45
        app()
        TIME.append(t1-t0)
        poppy.r_shoulder_x.goal_position = 5 #5
        app()
        TIME.append(t1-t0)
        poppy.r_arm_z.goal_position = 20 #20
        app()
        TIME.append(t1-t0)
        poppy.r_elbow_y.goal_position = -65 #-129
        
        app()
        TIME.append(t1-t0)

        time.sleep(rest)
        app()
        TIME.append(t1-t0)


In [61]:
runn = True
def alsa_rattle():
    while rattle_shake().is:
        l,data = inp.read()
        if l:
            # Return the maximum of the absolute value of all samples in a fragment.
            data_table.append(audioop.max(data,2))
            w.writeframes(data)
            # print audioop.max(data, 2)
        time.sleep(.001)
    rest_position()

In [57]:
for x in data_table:
	print(x)

In [56]:
alsa_rattle()
#rest_position()

In [22]:
for x in poppy.motors:
    print(x)

<DxlMotor name=r_shoulder_x id=52 pos=-0.13>
<DxlMotor name=r_shoulder_y id=51 pos=-0.31>
<DxlMotor name=r_arm_z id=53 pos=0.13>
<DxlMotor name=r_elbow_y id=54 pos=-0.13>


In [37]:
poppy.r_shoulder_y.goal_position = -45 #-10
poppy.r_shoulder_x.goal_position = 5  #0
poppy.r_arm_z.goal_position = 20; #0
poppy.r_elbow_y.goal_position = -65 #0

In [55]:
#rattle_shake()
rest_position()

In [51]:
# RUN at the end of working

pos = []
TIME = []
sys_load = []
speed = []
temp = []
volt = []
for m in poppy.motors:
    m.compliant = True
time.sleep(1)
poppy.stop_sync()
poppy.close()